<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---
**Source:**<br>
Dataset adapted from: Aditya Khosla, Nityananda Jayadevaprakash, Bangpeng Yao and Li Fei-Fei. Novel dataset for Fine-Grained Image Categorization. First Workshop on Fine-Grained Visual Categorization (FGVC), IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2011.<br>
Retrieved from http://vision.stanford.edu/aditya86/ImageNetDogs/


# Dog Breed Classification Competition Model Submission Guide

---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/dog_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Create training data objects 

# Extracting all filepaths iteratively...
import os 

base_path = 'dog_competition_data/train_images'
categories = ['Basset Hound', 'Cairn', 'Lhasa', 'Miniature Poodle', 
              'Golden Retriever', 'Siberian Husky']

# Load file paths to fnames list object...
fnames = []
for category in categories:
    dog_folder = os.path.join(base_path, category)
    file_names = os.listdir(dog_folder)
    full_path = [os.path.join(dog_folder, file_name) for file_name in file_names]
    fnames.append(full_path)

print('number of images for each category:', [len(f) for f in fnames])
print(fnames[0][0]) # Example of file name

# Create y data made up of correctly ordered labels from file folders...
from itertools import repeat

basset = list(repeat("Basset Hound", 140)) #i.e.: 140 filenames in Basset Hound folder
cairn = list(repeat("Cairn", 158))
lhasa = list(repeat("Lhasa", 149))
poodle = list(repeat("Miniature Poodle", 124))
golden = list(repeat("Golden Retriever", 120))
husky = list(repeat("Siberian Husky", 153))

# Combine into single list of y labels...
y_labels = basset + cairn + lhasa + poodle + golden + husky

# Need to one-hot encode for Keras. Let's use Pandas...
import pandas as pd
y_train = pd.get_dummies(y_labels)

number of images for each category: [140, 158, 149, 124, 120, 153]
dog_competition_data/train_images/Basset Hound/n02088238_11200.jpg


##2.   Preprocess data / Write and Save Preprocessor function


In [4]:
def preprocessor(data, shape=(192, 192)):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            data
                list of unprocessed images
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(data) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.


        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels].
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

In [5]:
# Use preprocessor to create X_train object 

# Import image, load to array of shape height, width, channels, then min/max transform...
# Read in all images from filenames...
import numpy as np 

preprocessed_image_data = [preprocessor(x) for x in fnames[0] + fnames[1] + fnames[2] + 
                           fnames[3] + fnames[4] + fnames[5]]

# Object needs to be an array rather than a list for Keras. (vstack converts above list to array object.)
X_train = np.vstack(preprocessed_image_data)
# Assigning to X_train to highlight that this represents feature input data for our model.

In [6]:
# Create X_test from numbered files

# Preprocess X_test image data to generate predictions from models 
import numpy as np

file_names = [('dog_competition_data/test_images/' + str(i) + '.jpg') for i in range(1, 212)]

preprocessed_image_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed images
X_test = np.vstack(preprocessed_image_data) 

##3. Fit model on preprocessed data and save preprocessor function and model 


In [7]:
# Let's build a convnet model...
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers

with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
                                 
        cnn1 = Sequential()
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=[192, 192, 3]))
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        cnn1.add(MaxPooling2D(pool_size=2)) 
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
        cnn1.add(MaxPooling2D(pool_size=2))
        cnn1.add(Dropout(0.3))
        cnn1.add(Flatten())
        cnn1.add(Dense(50, activation='relu'))
        cnn1.add(Dropout(0.2))
        cnn1.add(Dense(6, activation='softmax'))

        # Compile model...
        cnn1.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        # Fitting the NN to the Training set...
        hist = cnn1.fit(X_train, y_train, epochs=3, verbose=1, validation_split=0.2)

Epoch 1/3
22/22 [==============================] - 54s 2s/step - loss: 2.1143 - accuracy: 0.2059 - val_loss: 3.2304 - val_accuracy: 0.0000e+00
Epoch 2/3
22/22 [==============================] - 42s 2s/step - loss: 1.8930 - accuracy: 0.2548 - val_loss: 3.8129 - val_accuracy: 0.0000e+00
Epoch 3/3
22/22 [==============================] - 43s 2s/step - loss: 1.7580 - accuracy: 0.2948 - val_loss: 4.0398 - val_accuracy: 0.0237


#### Save preprocessor function to local "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [9]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(cnn1, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [10]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://o537m24bc5.execute-api.us-east-1.amazonaws.com/prod/m"#This is the unique rest api that powers this Dog Breed Classification Model Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [11]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=cnn1.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 3

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1313


In [13]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,conv2d_layers,maxpooling2d_layers,dropout_layers,flatten_layers,dense_layers,relu_act,softmax_act,loss,optimizer,model_config,memory_size,username,version
0,29.38%,22.72%,20.17%,30.47%,keras,False,True,Sequential,5,14164358,nan,nan,nan,1,4,3,1,function,RMSprop,"{'name': 'sequential_4', 'laye...",206888,AIModelShare,2
1,25.59%,21.25%,26.64%,24.48%,keras,False,True,Sequential,11,1851204,nan,2.000000,2.000000,1,2,5,1,function,RMSprop,"{'name': 'sequential_1', 'laye...",302920,AIModelShare,1
2,18.96%,12.51%,47.03%,18.50%,keras,False,True,Sequential,11,1850964,4.000000,2.000000,2.000000,1,2,5,1,str,RMSprop,"{'name': 'sequential', 'layers...",2730456,AIModelShare,3


## 5. Repeat submission process to improve place on leaderboard


In [14]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).

with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
      cnn2 = Sequential()
      cnn2.add(Dense(128,  input_shape=(192, 192, 3), activation='relu'))
      cnn2.add(Dense(64, activation='relu'))
      cnn2.add(Dense(64, activation='relu'))
      cnn2.add(Flatten())
      cnn2.add(Dense(6, activation='softmax')) 
                                                  
      # Compile model
      cnn2.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

      # Fitting the NN to the Training set
      cnn2.fit(X_train, y_train, 
                    epochs = 3, verbose=1, validation_split=.2)

Epoch 1/3
22/22 [==============================] - 88s 4s/step - loss: 25.3183 - accuracy: 0.2430 - val_loss: 12.0798 - val_accuracy: 0.0059
Epoch 2/3
22/22 [==============================] - 83s 4s/step - loss: 1.5209 - accuracy: 0.3748 - val_loss: 13.3431 - val_accuracy: 0.0947
Epoch 3/3
22/22 [==============================] - 83s 4s/step - loss: 1.7190 - accuracy: 0.3881 - val_loss: 9.3149 - val_accuracy: 0.0355


In [15]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(cnn2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [16]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
prediction_column_index=cnn2.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 4

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1313


In [17]:
# Compare two or more models
data=mycompetition.compare_models([3,4], verbose=1)
mycompetition.stylize_compare(data)

,Model_3_Layer,Model_3_Shape,Model_3_Params,Model_4_Layer,Model_4_Shape,Model_4_Params
0,Conv2D,"[None, 192, 192, 16]",448,Dense,"[None, 192, 192, 128]",512.000000
1,Conv2D,"[None, 192, 192, 16]",2320,Dense,"[None, 192, 192, 64]",8256.000000
2,MaxPooling2D,"[None, 96, 96, 16]",0,Dense,"[None, 192, 192, 64]",4160.000000
3,Conv2D,"[None, 96, 96, 16]",2320,Flatten,"[None, 2359296]",0.000000
4,Conv2D,"[None, 96, 96, 16]",2320,Dense,"[None, 6]",14155782.000000
5,MaxPooling2D,"[None, 48, 48, 16]",0,None,None,nan
6,Dropout,"[None, 48, 48, 16]",0,None,None,nan
7,Flatten,"[None, 36864]",0,None,None,nan
8,Dense,"[None, 50]",1843250,None,None,nan
9,Dropout,"[None, 50]",0,None,None,nan


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [19]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [20]:
import keras_tuner as kt
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers

#Define model structure & parameter search space with function
def build_model(hp):
    with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
                                 
        model = Sequential()
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=[192, 192, 3]))
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        model.add(MaxPooling2D(pool_size=2)) 
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        model.add(Conv2D(filters=hp.Int("filters", min_value=8, max_value=32, step=8), #range 8-32 inclusive, minimum step between tested values is 8
                         kernel_size=3, padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.3))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(6, activation='softmax'))

        # Compile model...
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        return model 

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=2, #max number of trials to run during search
    executions_per_trial=1, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(x_train_split, y_train_split, epochs=1, validation_data=(x_val, y_val))


Trial 2 Complete [00h 01m 23s]
val_accuracy: 0.15976330637931824

Best val_accuracy So Far: 0.20710058510303497
Total elapsed time: 00h 02m 29s


In [21]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=X_train, y=y_train, epochs=1)


27/27 [==============================] - 56s 2s/step - loss: 2.9431 - accuracy: 0.1919


In [22]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [23]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 5

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1313


In [24]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,conv2d_layers,maxpooling2d_layers,dropout_layers,flatten_layers,dense_layers,relu_act,softmax_act,loss,optimizer,model_config,memory_size,username,version
0,29.38%,22.72%,20.17%,30.47%,keras,False,True,Sequential,5,14164358,nan,nan,nan,1,4,3,1,function,RMSprop,"{'name': 'sequential_4', 'laye...",206888,AIModelShare,2
1,25.59%,21.25%,26.64%,24.48%,keras,False,True,Sequential,11,1851204,nan,2.000000,2.000000,1,2,5,1,function,RMSprop,"{'name': 'sequential_1', 'laye...",302920,AIModelShare,1
2,23.70%,21.28%,19.68%,24.57%,keras,False,True,Sequential,5,14168710,nan,nan,nan,1,4,3,1,str,RMSprop,"{'name': 'sequential_1', 'laye...",2339576,AIModelShare,4
3,18.96%,12.51%,47.03%,18.50%,keras,False,True,Sequential,11,1850964,4.000000,2.000000,2.000000,1,2,5,1,str,RMSprop,"{'name': 'sequential', 'layers...",2730456,AIModelShare,3
4,18.01%,9.47%,16.96%,16.50%,keras,False,True,Sequential,11,1850964,4.000000,2.000000,2.000000,1,2,5,1,str,RMSprop,"{'name': 'sequential_1', 'laye...",2400352,AIModelShare,5


In [25]:
# Compare two or more models
data=mycompetition.compare_models([4, 5], verbose=1)
mycompetition.stylize_compare(data)

,Model_4_Layer,Model_4_Shape,Model_4_Params,Model_5_Layer,Model_5_Shape,Model_5_Params
0,Dense,"[None, 192, 192, 128]",512.000000,Conv2D,"[None, 192, 192, 16]",448
1,Dense,"[None, 192, 192, 64]",8256.000000,Conv2D,"[None, 192, 192, 16]",2320
2,Dense,"[None, 192, 192, 64]",4160.000000,MaxPooling2D,"[None, 96, 96, 16]",0
3,Flatten,"[None, 2359296]",0.000000,Conv2D,"[None, 96, 96, 16]",2320
4,Dense,"[None, 6]",14155782.000000,Conv2D,"[None, 96, 96, 16]",2320
5,None,None,nan,MaxPooling2D,"[None, 48, 48, 16]",0
6,None,None,nan,Dropout,"[None, 48, 48, 16]",0
7,None,None,nan,Flatten,"[None, 36864]",0
8,None,None,nan,Dense,"[None, 50]",1843250
9,None,None,nan,Dropout,"[None, 50]",0
